In [2]:
# from skimage.util.montage import montage2d
import matplotlib.pyplot as plt
import nibabel as nib
import pandas as pd
import numpy as np
import skimage
import time
import sys
import os

from skimage import measure
from shutil import move
from PIL import Image, ImageDraw
from scipy.spatial import ConvexHull
from scipy.spatial.qhull import QhullError
from sklearn.model_selection import train_test_split

In [3]:
print(skimage.__version__)

0.18.1


In [4]:
home = os.path.expanduser('~')
directory = os.path.join('Datasets', 'ImageCLEF')
filename = '4231cdb3-af46-4674-be08-95b904a62093_TrainSet_metaData.csv'
path = os.path.join(home, directory, filename)

dataset_dir = os.path.join(home, directory, 'Dataset')

df = pd.read_csv(path)

fnames = df['FileName'].to_list()
labels = df['TypeOfTB'].to_list()

idxs = [i for i in range(len(labels))]

train_idxs, test_idxs = train_test_split(
                            idxs, 
                            test_size=0.2, 
                            random_state=42, 
                            shuffle=True, 
                            stratify=labels)

In [5]:
img_depth = 84

def read_nifti_file(filepath):
    """Read and load volume"""
    # Read file
    scan = nib.load(filepath)
    # Get raw data
    scan = scan.get_fdata()
    return scan

def normalize(volume, min, max):
    """Normalize the volume"""
    
    level = 250
    window = 50
#     max = int(level+ (window/2))
#     min = int(level- (window/2))
    
#     min = -800
#     max = -300
#     volume[volume < min] = min
#     volume[volume > max] = max
    volume = volume.clip(min, max)

#     volume = (volume - min) / (max - min)
    volume = volume.astype("float32")
    '''
    
    level = 1000
    window = 250
    max = level+ (window/2)
    max = level- (window/2)
    
    volume = volume.clip(min, max)
    volume = volume.astype("float32")
    '''
    
    return volume
'''
def intensity_seg(ct_numpy, min=-1000, max=-300):
    clipped = clip_ct(ct_numpy, min, max)
    return measure.find_contours(ct_numpy, 0.95)
'''    
def intensity_seg(ct_numpy, min=-1000, max=-300):
    clipped = ct_numpy.clip(min, max)
    clipped[clipped != max] = 1
    clipped[clipped == max] = 0
    return measure.find_contours(clipped, 0.95)
    
def contour_distance(contour):
    """
    Given a set of points that may describe a contour
     it calculates the distance between the first and the last point
     to infer if the set is closed.
    Args:
        contour: np array of x and y points

    Returns: euclidean distance of first and last point
    """
    dx = contour[0, 1] - contour[-1, 1]
    dy = contour[0, 0] - contour[-1, 0]
    return np.sqrt(np.power(dx, 2) + np.power(dy, 2))

def set_is_closed(contour):
    if contour_distance(contour) < 1:
        return True
    else:
        return False
    
def find_lungs(contours):
    """
    Chooses the contours that correspond to the lungs and the body
    First, we exclude non-closed sets-contours
    Then we assume some min area and volume to exclude small contours
    Then the body is excluded as the highest volume closed set
    The remaining areas correspond to the lungs
    Args:
        contours: all the detected contours
    Returns: contours that correspond to the lung area
    """
    body_and_lung_contours = []
    vol_contours = []
    for contour in contours:
        hull = ConvexHull(contour)
       # set some constraints for the volume
        if hull.volume > 2000 and set_is_closed(contour):
            body_and_lung_contours.append(contour)
            vol_contours.append(hull.volume)
    # Discard body contour
    if len(body_and_lung_contours) == 2:
        return body_and_lung_contours
    elif len(body_and_lung_contours) > 2:
        vol_contours, body_and_lung_contours = (list(t) for t in 
                zip(*sorted(zip(vol_contours, body_and_lung_contours))))
        body_and_lung_contours.pop(-1) # body is out!
    return body_and_lung_contours # only lungs left !!!

def create_mask_from_polygon(image, contours):
    """
    Creates a binary mask with the dimensions of the image and
    converts the list of polygon-contours to binary masks and merges them together
    Args:
        image: the image that the contours refer to
        contours: list of contours
    Returns:
    """
    lung_mask = np.array(Image.new('L', image.shape, 0))
    for contour in contours:
        x = contour[:, 0]
        y = contour[:, 1]
        polygon_tuple = list(zip(x, y))
        img = Image.new('L', image.shape, 0)
        ImageDraw.Draw(img).polygon(polygon_tuple, outline=0, fill=1)
        mask = np.array(img)
        lung_mask += mask
    lung_mask[lung_mask > 1] = 1  # sanity check to make 100% sure that the mask is binary
    return lung_mask.T  # transpose it to be aligned with the image dims

def create_vessel_mask(lung_mask, ct_numpy, denoise=False):
    vessels = lung_mask * ct_numpy  # isolate lung area
    vessels[vessels == 0] = -1000
    vessels[vessels >= -600] = 1
    vessels[vessels < -600] = 0
#     show_slice(vessels)
    if denoise:
        return denoise_vessels(lungs_contour, vessels)
#     show_slice(vessels)
    return vessels

def euclidean_dist(dx, dy):
    return np.sqrt(np.power(dx, 2) + np.power(dy, 2))

def denoise_vessels(lung_contour, vessels):
    vessels_coords_x, vessels_coords_y = np.nonzero(vessels)  # get non zero coordinates
    for contour in lung_contour:
        x_points, y_points = contour[:, 0], contour[:, 1]
        for (coord_x, coord_y) in zip(vessels_coords_x, vessels_coords_y):
            for (x, y) in zip(x_points, y_points):
                d = euclidean_dist(x - coord_x, y - coord_y)
                if d <= 0.1:
                    vessels[coord_x, coord_y] = 0
    return vessels

def show_slice(slice):
    """
    Function to display an image slice
    Input is a numpy 2D array
    """
    plt.figure()
    plt.imshow(slice.T, cmap="gray", origin="lower")

def resize_volume(img):
    """Resize across z-axis"""
    # Set the desired depth
    
#     print(f"Shape: {img.shape}")
    desired_depth = img_depth
    desired_width = 512
    desired_height = 512
    # Get current depth
    current_depth = img.shape[-1]
    current_width = img.shape[0]
    current_height = img.shape[1]
    # Compute depth factor
    depth = current_depth / desired_depth
    width = current_width / desired_width
    height = current_height / desired_height
    depth_factor = 1 / depth
    width_factor = 1 / width
    height_factor = 1 / height
    # Rotate
    
#     img = ndimage.rotate(img, 90, reshape=False)
    # Resize across z-axis
    
    img = ndimage.zoom(img, (width_factor, height_factor, depth_factor), order=1)
    return img

def show_contour(image, contours, name=None, save=False):
    fig, ax = plt.subplots()
    ax.imshow(image.T, cmap=plt.cm.gray)
    for contour in contours:
        ax.plot(contour[:, 0], contour[:, 1], linewidth=1)

    ax.set_xticks([])
    ax.set_yticks([])

    if save:
        plt.savefig(name)
        plt.close(fig)
    else:
        plt.show()

def process_scan(path, min=-1000, max=-150):
    """Read and resize volume"""
    # Read scan
    volume = read_nifti_file(path)
    
    print(path)
    print(volume.shape)
    
    slices = volume.shape[-1]
    sl_min = int((2/9) * slices + (14/9))
    sl_max = int((7/54) * slices + (49/54))
    
    volume = volume[:, :, sl_min:-sl_max]
    
    volume = np.mean(volume, axis=2)
    
    print(f'Min {sl_min}\tMax {sl_max}')
    
    contour = intensity_seg(volume, min=-1000, max=-300)
    contour = find_lungs(contour)
    
    mask = create_mask_from_polygon(volume, contour)
    
    vessels = create_vessel_mask(mask, volume, lungs_contour=contour, denoise=False)
    
    return volume

def overlay_plot(im, mask):
    plt.figure()
    plt.imshow(im.T, 'gray', interpolation='none')
    plt.imshow(mask.T, 'jet', interpolation='none', alpha=0.5)

In [6]:
def get_mean_mask(img_name):
    ct_img = nib.load(os.path.join(dataset_dir, img_name))
    ct_numpy = ct_img.get_fdata()

    volume = ct_numpy

    slices = volume.shape[-1]
    sl_min = int((2/9) * slices + (14/9))
    sl_max = int((7/54) * slices + (49/54))
    
    volume = volume[:, :, sl_min:-sl_max]

    slices_arr = []

    for i in range(volume.shape[2]):
        slice_i = volume[:, :, i]
    
        contours = intensity_seg(slice_i, -1000, -300)

        lungs_contour = find_lungs(contours)
        lung_mask = create_mask_from_polygon(slice_i, lungs_contour)

        vessels_only = create_vessel_mask(lung_mask, slice_i, denoise=False)
    
        slices_arr.append(vessels_only)
    
    slices_arr = np.array(slices_arr)
    slices_arr_mean = np.mean(slices_arr, axis=0)

#     min_t, max_t = 0.033, 0.2current
    min_t, max_t = 0.033, 0.135

    slices_arr_mean[slices_arr_mean < min_t] = 0
    slices_arr_mean[slices_arr_mean > max_t] = 1
    
    return slices_arr_mean

In [7]:
def crop_image(image_array):
    image = Image.fromarray(image_array)
    left, upper, right, lower = 110, 0, 512, 512
    image = image.crop((left, upper, right, lower))
#     plt.imshow
    width, height = image.size
#     print(f'Width: {width}\tHeight: {height}')
    return image

In [21]:
save_path = os.path.join(home, directory, 'Mean_Slice_Masks_331')

try:
    os.makedirs(save_path)
except FileExistsError as err:
    print(f'{err}')
    print(f'It is recommended you empty the directory before creating masks')

In [24]:
masks_list = os.listdir(save_path)
img_names = os.listdir(dataset_dir)

# (left, upper, right, lower)

for name in img_names:
    name = name.split('.')
    if name[1] != 'nii' or name[2] != 'gz':
        continue
        
    sl_name = f'{name[0]}_Mean_Slice_Mask.png'

    img_name = f'{name[0]}_Mean_Slice_Mask.png'
    img_save_path = os.path.join(save_path, img_name)
    
    if sl_name in masks_list:
        print(f'EXISTS: {img_save_path}')
        continue
    
    try:
        mean_mask = get_mean_mask(f'{name[0]}.nii.gz')

        image = crop_image(mean_mask)
        
        img_arr = np.asarray(image)
        black = np.zeros(shape=(512, 100))
        img_arr = np.append(black, img_arr, axis=1)

#         plt.imshow(img_arr, cmap='gray')
        image = Image.fromarray(img_arr)
        image = image.resize(size=(331, 331), resample=Image.LANCZOS)
#         image_3channel = Image.new('RGB', image.size)
#         image_3channel.paste(image)
        
#         plt.imshow(np.asarray(image_3channel))
        image = image.convert(mode='I')
        image.save(img_save_path)
        print(f'CREATED: {img_save_path}')
#         break
        
    except QhullError as err:
        print(f'ERROR creating: {img_save_path} \n{err}\n')
    

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0001_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0002_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0003_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0004_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0005_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0006_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0007_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0008_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0009_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0010_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0084_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0085_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0086_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0087_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0088_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0089_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0090_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0091_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0092_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0093_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0167_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0168_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0169_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0170_Mean_Slice_Mask.png
ERROR creating: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0171_Mean_Slice_Mask.png 
QH6214 qhull input error: not enough points(2) to construct initial simplex (need 3)

While executing:  | qhull i Qt
Options selected for Qhull 2019.1.r 2019/06/21:
  run-id 1924926181  incidence  Qtriangulate  _pre-merge  _zero-centrum
  _maxoutside  0


CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0172_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0173_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/Imag

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0248_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0249_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0250_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0251_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0252_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0253_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0254_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0255_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0256_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0257_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0332_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0333_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0334_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0335_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0336_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0337_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0338_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0339_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0340_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0341_Mean_Slice_Mask.png
CREATED: /

...

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0526_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0527_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0528_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0529_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0530_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0531_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0532_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0533_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0534_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0535_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0609_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0610_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0611_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0612_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0613_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0614_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0615_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0616_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0617_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0618_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0692_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0693_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0694_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0695_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0696_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0697_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0698_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0699_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0700_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0701_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0775_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0776_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0777_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0778_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0779_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0780_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0781_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0782_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0783_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0784_Mean_Slice_Mask.png
CREATED: /

CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0858_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0859_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0860_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0861_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0862_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0863_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0864_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0865_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0866_Mean_Slice_Mask.png
CREATED: /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0867_Mean_Slice_Mask.png
CREATED: /

In [25]:
def move_files(idxs_list, dset_type):
    base_new = os.path.join(save_path, dset_type)
    try:
        for i in range(1, 5+1):
            os.makedirs(os.path.join(base_new, f'{i}'))
    except FileExistsError as err:
        print('Error: Directories already exist. Aborting')
        sys.exit()
    count = 0
    
    for idx in idxs_list:
        image_class = str(labels[idx])
        
        from_dir = save_path
        to_dir = os.path.join(base_new, image_class)

        base_fname = fnames[idx].split('.')[0]
        
        image_name = f'{base_fname}_Mean_Slice_Mask.png'
            
        origin = os.path.join(from_dir, image_name)
        destination = os.path.join(to_dir, image_name)
            
        try:
            move(origin, destination)
            print(f'{origin} --> {destination}')
            count += 1
        except FileNotFoundError as err:
            print("LOG ERROR: {err}")
                
    return count

t0 = time.monotonic()
train_count = move_files(train_idxs, 'train')    

test_count = move_files(test_idxs, 'test')
t_total = time.monotonic() - t0

print(f"\n\n{'='*125}\n")
print(f"""
Moved {train_count} training images and {test_count} test images in {t_total: 0.2f} seconds.
\n\n""")

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0377_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0377_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0092_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0092_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0181_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0181_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0020_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0020_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0247_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0247_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0754_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/3/TRN_0754_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0105_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0105_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0565_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0565_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0725_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/3/TRN_0725_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0509_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0509_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0716_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/3/TRN_0716_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0824_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/4/TRN_0824_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0096_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0096_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0114_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0114_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0606_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0606_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0385_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0385_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0430_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0430_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0550_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0550_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0425_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0425_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0722_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/3/TRN_0722_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0041_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0041_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0021_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0021_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0234_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0234_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0123_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0123_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0438_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0438_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

...

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0859_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/5/TRN_0859_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0416_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0416_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0793_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/4/TRN_0793_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0718_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/3/TRN_0718_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0532_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0532_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0519_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/2/TRN_0519_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0368_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0368_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0100_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0100_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0781_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/4/TRN_0781_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0168_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/train/1/TRN_0168_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0642_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/2/TRN_0642_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0685_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/3/TRN_0685_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0865_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/5/TRN_0865_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0570_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/2/TRN_0570_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0324_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0324_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0749_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/3/TRN_0749_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0453_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0453_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0366_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0366_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0676_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/2/TRN_0676_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0414_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0414_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_

/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0629_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/2/TRN_0629_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0346_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0346_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0396_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0396_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0890_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/5/TRN_0890_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/TRN_0121_Mean_Slice_Mask.png --> /home/ayushman.singh/Datasets/ImageCLEF/Mean_Slice_Masks_331/test/1/TRN_0121_Mean_Slice_Mask.png
/home/ayushman.singh/Datasets/ImageCLEF/Mean_